#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 1:  Basic Machine Learning + Learning to Rank 

### 100 points [5% of your final grade]

### Due: Monday, February 12 by 11:59pm

*Goals of this homework:* In this homework you will get hands-on experience with (i) the basics of machine learning (e.g. train/test data, cross-validation, different classifiers) and interpreting results; and (ii) learning to rank.

*Submission Instructions:* To submit your homework, rename this notebook as UIN_hw#.ipynb. For example, this homework submission would be: YourUIN_hw1.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Thursday, February 15 at 11:59pm.

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1: Basics of ML (70 points)

For this part, we're going to get familiar with scikit-learn (a great ML toolkit that is very popular) and the major issues in training a model, testing it, and interpreting the results. Our goal in this assignment is to build a classifier to determine if a Yelp review is "food-relevant" or not.

## Dataset: Yelp review data

First, you will need to download the training_data.json file from the Resources tab on Piazza, a collection of 40,000 json-encoded Yelp reviews we sampled from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge).

You'll see that each line corresponds to a review on a particular business. The label (class) information of each review is in the "label" field. It is **either "Food-relevant" or "Food-irrelevant"**.

## Part 1.1: Parsing Yelp (15 points)

For this first part, we will build a parser for extracting tokens from the **review text** only. First, you should tokenize each review using **whitespaces and punctuations as delimiters**. Do not remove stopwords. You should apply casefolding (lower case everything) and use the [nltk Porter stemmer](http://www.nltk.org/api/nltk.stem.html#module-nltk.stem.porter) ... you may need to install nltk if you don't have it already. 

In [2]:
# your code here
# use as many cells as you need
print 'starting'

starting


### Unique tokens?

Once you have your parser working, you should report here the size of your feature space. That is, how many unique tokens do you find?

In [3]:
import json
import string
import operator
import nltk
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import svm
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.datasets import make_classification
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix



file = 'F:/SEM-2/IR/HW_1/train.json'
st = PorterStemmer()
words = []
lines = []
labels = []

print'started'
with open(file) as f:
    for line in f:
        #print(line)
        json_data = json.loads(line.lower())
        text = json_data['text']
        lines.append(text)
        label = json_data['label']
        #print(label)
        labels.append(label)
        #print(json_data)
        list = re.findall('\w+', text)
        #print list
        #words.append(list)
        for temp in list:
            #print temp, st.stem(temp)
            words.append(st.stem(temp))
#print words
frequency = {}

for word in words:
    count = frequency.get(word,0)
    frequency[word] = count + 1
     
frequency_list = frequency.keys()
print len(frequency_list)

started
36555


### The Most Popular Words

Great, now we can tokenize the documents. Let's make a list of the most popular words in our reviews. For this step, you should maintain a count of how many times each word occurs. Then you should print out the top-20 words in your reviews.

Your output should look like this:

Rank Token Count

1 awesome 78

... ...

In [4]:
freq_sorted = sorted(frequency, key=frequency.get, reverse=True)
for r in freq_sorted[:20]:
    print r, frequency[r]

the 246309
i 168931
and 168589
a 134904
to 128139
it 78867
of 76237
wa 74020
is 63496
for 60867
in 60523
that 50804
my 50565
you 45881
they 43635
thi 39940
with 39340
have 39082
but 37967
on 35388


### Zipf's Law

Recall in class our discussion of Zipf's law. Let's see if this law applies to our Yelp reviews. You should use matplotlib to plot the log-base10 term counts on the y-axis versus the log-base10 rank on the x-axis. Your aim is to create a figure like the one in Figure 5.2 of the textbook.

In [5]:
# your code here
print 'sss'

sss


What do you observe? Is this consistent with Zipf's law?

*Your answer goes here*

## Part 1.2: Feature Represenation (10 points)

In this part you will build feature vectors for each review. This will be input to our ML classifiers. You should call your parser from earlier, using all the same assumptions (e.g., casefolding, stemming). Each feature value should be the term count for that review.

In [6]:
print 'start'
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lines)
print 'done'

start
done


## Part 1.3: Machine Learning Basics (30 points)

In this part you will evaluate a bunch of classifiers -- kNN, Decision tree, Naive Bayes, and SVM -- on the feature vectors generated in the previous task in two different settings. **You do not need to implement any classifier from scratch. You may use scikit-learn's built-in capabilities.**

### Setting 1: Splitting data into train-test 

In the first setting, you should treat the first 70% of your data as training. The remaining 30% should be for testing. 

### Setting 2: Using 5 fold cross-validation

In the second setting, use 5-folk cross-validation. 

### What to report

* Report the overall accuracy for both settings.
* For the class "Food-relevant", report the precision and recall for both settings.
* For the class "Food-irrelevant", report the precision and recall for both settings.

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split


print 'split'

X_train, X_test, y_train, y_test = train_test_split(X, np.transpose(labels), test_size=0.3, random_state=42)

print 'svm'
svm_clf = svm.SVC(kernel ='linear', C = 1.0)
svm_clf.fit(X_train,y_train)
svm_pred = svm_clf.predict(X_test)

print 'knn'
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)
knn_pred = knn_clf.predict(X_test)

print 'tree'
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)
tree_pred = tree_clf.predict(X_test)

print 'nb'
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)
nb_pred = nb_clf.predict(X_test)

print 'accuracy with splitting data------------------------------'
print 'svm',accuracy_score(y_test, svm_pred)
print 'knn',accuracy_score(y_test, knn_pred)
print 'decision tree',accuracy_score(y_test, tree_pred)
print 'naive bayes',accuracy_score(y_test, nb_pred)

print 'precision and recall is only meaningful if they are calculated on the test data'
print 'so they are calculated on test data and for svm classifier'

target_names = ['relevant', 'irrelevant']
print(classification_report(y_test, svm_pred, target_names=target_names))

y = np.transpose(labels)
skf = StratifiedKFold(y, n_folds=5)
for train_index, test_index in skf:
    print("TRAIN:", train_index, "TEST:", test_index)
    XK_train, XK_test = X[train_index], X[test_index]
    yK_train, yK_test = y[train_index], y[test_index]

svmK_clf = svm.SVC(kernel ='linear', C = 1.0)
svmK_clf.fit(XK_train,yK_train)
svmK_pred = svmK_clf.predict(XK_test)

knnK_clf = KNeighborsClassifier(n_neighbors=3)
knnK_clf.fit(XK_train, yK_train)
knnK_pred = knnK_clf.predict(XK_test)

treeK_clf = DecisionTreeClassifier()
treeK_clf.fit(XK_train, yK_train)
treeK_pred = treeK_clf.predict(XK_test)

nbK_clf = MultinomialNB()
nbK_clf.fit(XK_train, yK_train)
nbK_pred = nbK_clf.predict(XK_test)

print 'accuracy with 5 fold cross validation------------------------------'
print 'svm',accuracy_score(yK_test, svmK_pred)
print 'knn',accuracy_score(yK_test, knnK_pred)
print 'tree',accuracy_score(yK_test, treeK_pred)
print 'naive bayes',accuracy_score(yK_test, nbK_pred)

print 'precision and recall is only meaningful if they are calculated on the test data'
print 'so they are calculated on test data and for svm classifier'

target_names = ['relevant', 'irrelevant']
print(classification_report(yK_test, svmK_pred, target_names=target_names))


split
svm
knn
tree
nb
accuracy with splitting data------------------------------
svm 0.933083333333
knn 0.706083333333
decision tree 0.88375
naive bayes 0.948083333333
precision and recall is only meaningful if they are calculated on the test data
so they are calculated on test data and for svm classifier
             precision    recall  f1-score   support

   relevant       0.94      0.93      0.93      5999
 irrelevant       0.93      0.94      0.93      6001

avg / total       0.93      0.93      0.93     12000

('TRAIN:', array([ 5451,  5452,  5453, ..., 39997, 39998, 39999]), 'TEST:', array([    0,     1,     2, ..., 13245, 13246, 13247]))
('TRAIN:', array([    0,     1,     2, ..., 39997, 39998, 39999]), 'TEST:', array([ 5451,  5452,  5453, ..., 27312, 27313, 27314]))
('TRAIN:', array([    0,     1,     2, ..., 39997, 39998, 39999]), 'TEST:', array([11513, 11514, 11515, ..., 31997, 31998, 31999]))
('TRAIN:', array([    0,     1,     2, ..., 39997, 39998, 39999]), 'TEST:', array(

## Part 1.4: Analyzing your results (5 points) 

OK, now that you have tried four different classifiers, what do you observe? Any conclusions you can draw? Give us one or two paragraphs summarizing your findings.

So we have implemented 4 classifiers, and seeing the result we can say that naive bayes is the best classifier. Maybe its because of the assumption it makes that the probability of occurrence of any word given the class label, is independent of the probability of occurrence of any other word, given that label.

Also sophasticated methods like svm, knn works at their best when they are used for multiclass classification. 
On how we can improve the classifier is by check the frequencies of semantically similar words. For example if the review of the certain movie is good, it generally contains words which mean good and are praise-like. Semantic grouping of words and classification on the basis of that can help. Also the use of frequency of negative words can be used as a feature.

## Part 1.5: Improving your classifier (10 points)

I think we can do better! In this part, your job is to create new features that you can think can help improve your classifier. You may choose to use new weightings for your words, new derived features (e.g., count of 3-letter words), or whatever you like. You may also add in the extra features in the json: funny, useful, cool. You will need to experiment with different approaches ... once you finalize on your best approach, include the features here with a description (that is, tell us what the feature means). Then give us your classifier results!

In [8]:
# your code here ... add as many cells as you need for features, results, and discussion.
print "we can use TFIDF"

from sklearn.feature_extraction.text import TfidfVectorizer
print 'start'
Tvectorizer = TfidfVectorizer()
XT = Tvectorizer.fit_transform(lines)
print 'done'



from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split


print 'split'

XT_train, XT_test, yT_train, yT_test = train_test_split(XT, np.transpose(labels), test_size=0.3, random_state=42)

print 'svm'
svm_clf = svm.SVC(kernel ='linear', C = 1.0)
svm_clf.fit(XT_train,yT_train)
svm_pred = svm_clf.predict(XT_test)

print 'knn'
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(XT_train, yT_train)
knn_pred = knn_clf.predict(XT_test)

print 'tree'
tree_clf = DecisionTreeClassifier()
tree_clf.fit(XT_train, yT_train)
tree_pred = tree_clf.predict(XT_test)

print 'NB'
nb_clf = MultinomialNB()
nb_clf.fit(XT_train, yT_train)
nb_pred = nb_clf.predict(XT_test)

print 'accuracy with splitting data------------------------------'
print 'svm',accuracy_score(yT_test, svm_pred)
print 'knn',accuracy_score(yT_test, knn_pred)
print 'decision tree',accuracy_score(yT_test, tree_pred)
print 'naive bayes',accuracy_score(yT_test, nb_pred)

print 'precision and recall is only meaningful if they are calculated on the test data'
print 'so they are calculated on test data and for svm classifier'

target_names = ['relevant', 'irrelevant']
print(classification_report(y_test, svm_pred, target_names=target_names))

y = np.transpose(labels)
skf = StratifiedKFold(y, n_folds=5)
for train_index, test_index in skf:
    #print("TRAIN:", train_index, "TEST:", test_index)
    XK_train, XK_test = XT[train_index], XT[test_index]
    yK_train, yK_test = y[train_index], y[test_index]

svmK_clf = svm.SVC(kernel ='linear', C = 1.0)
svmK_clf.fit(XK_train,yK_train)
svmK_pred = svmK_clf.predict(XK_test)

knnK_clf = KNeighborsClassifier(n_neighbors=3)
knnK_clf.fit(XK_train, yK_train)
knnK_pred = knnK_clf.predict(XK_test)

treeK_clf = DecisionTreeClassifier()
treeK_clf.fit(XK_train, yK_train)
treeK_pred = treeK_clf.predict(XK_test)

nbK_clf = MultinomialNB()
nbK_clf.fit(XK_train, yK_train)
nbK_pred = nbK_clf.predict(XK_test)

print 'accuracy with 5 fold cross validation------------------------------'
print 'svm',accuracy_score(yK_test, svmK_pred)
print 'knn',accuracy_score(yK_test, knnK_pred)
print 'tree',accuracy_score(yK_test, treeK_pred)
print 'naive bayes',accuracy_score(yK_test, nbK_pred)

print 'precision and recall is only meaningful if they are calculated on the test data'
print 'so they are calculated on test data and for svm classifier'

target_names = ['relevant', 'irrelevant']
print(classification_report(yK_test, svmK_pred, target_names=target_names))

we can use TFIDF
start
done
split
svm
knn
tree
NB
accuracy with splitting data------------------------------
svm 0.958416666667
knn 0.512166666667
decision tree 0.876416666667
naive bayes 0.950083333333
precision and recall is only meaningful if they are calculated on the test data
so they are calculated on test data and for svm classifier
             precision    recall  f1-score   support

   relevant       0.95      0.96      0.96      5999
 irrelevant       0.96      0.95      0.96      6001

avg / total       0.96      0.96      0.96     12000

accuracy with 5 fold cross validation------------------------------
svm 0.954625
knn 0.869
tree 0.882
naive bayes 0.958
precision and recall is only meaningful if they are calculated on the test data
so they are calculated on test data and for svm classifier
             precision    recall  f1-score   support

   relevant       0.94      0.97      0.96      4000
 irrelevant       0.97      0.94      0.95      4000

avg / total       0.96 

### BONUS: What are the most informative features in distinguishing these two classes?

In [ ]:
# Your code here

# Part 2: Learning to Rank (30 points)

For this part, we're going to play with some Microsoft LETOR data that has query-document relevance judgments. Let's see how learning to rank works in practice. 

First, you will need to download the MQ2008.zip file from the Resources tab on Piazza. This is data from the [Microsoft Research IR Group](https://www.microsoft.com/en-us/research/project/letor-learning-rank-information-retrieval/).

The data includes 15,211 rows. Each row is a query-document pair. The first column is a relevance label of this pair (0,1 or 2--> the higher value the more related), the second column is query id, the following columns are features, and the end of the row is comment about the pair, including id of the document. A query-document pair is represented by a 46-dimensional feature vector. Features are a numeric value describing a document and query such as TFIDF, BM25, Page Rank, .... You can find compelete description of features from [here](https://arxiv.org/ftp/arxiv/papers/1306/1306.2597.pdf).

The good news for you is the dataset is ready for analysis: It has already been split into 5 folds (see the five folders called Fold1, ..., Fold5).

For this assignment, we're going to leave our favorite scikit-learn and instead use [SVM-rank](https://www.cs.cornell.edu/people/tj/svm_light/svm_rank.html). This is the basic ranking SVM we talked about in class. You'll see that SVM-rank considers pairwise relevance between docs -- so based on the training data it will transform the data into pairs -- like D1 > D2 and then learn a separator.


## Part 2.1: Optimizing SVM-Rank (15 points)

First, you should explore how the different parameters affect the quality of the Ranking SVM. You'll see that you can vary the kernel function, the loss function and so forth. 

You should run SVM-Rank using the default options over each of the five folds. You should find the error on the test set (for example, depending on your settings, svm_rank_classify will give you the zero/one error statistics (that is, the number of correct pairs and the number of incorrect pairs). Report the average. 

Then try different parameters and report how they impact the quality of results. 

  Fold 1
 c = default  Zero/one-error on test set: 58.97% (64 correct, 92 incorrect, 156 total)
 c = 10       Zero/one-error on test set: 58.97% (64 correct, 92 incorrect, 156 total)
 c = 5, l = 2 Zero/one-error on test set: 58.33% (65 correct, 91 incorrect, 156 total)
 c = 3, w = 2 Zero/one-error on test set: 57.69% (66 correct, 90 incorrect, 156 total)
 c = 3, w = 4 Zero/one-error on test set: 57.05% (67 correct, 89 incorrect, 156 total)
 
 Fold 2
 c = default  Zero/one-error on test set: 56.69% (68 correct, 89 incorrect, 157 total)
 c = 10       Zero/one-error on test set: 57.32% (67 correct, 90 incorrect, 157 total)
 c = 5, l = 2 Zero/one-error on test set: 54.78% (71 correct, 86 incorrect, 157 total)
 c = 3, w = 2 Zero/one-error on test set: 56.05% (69 correct, 88 incorrect, 157 total)
 c = 3, w = 4 Zero/one-error on test set: 56.69% (68 correct, 89 incorrect, 157 total)
 
 Fold 3
 c = default  Zero/one-error on test set: 62.42% (59 correct, 98 incorrect, 157 total)
 c = 10       Zero/one-error on test set: 61.78% (60 correct, 97 incorrect, 157 total)
 c = 5, l = 2 Zero/one-error on test set: 61.15% (61 correct, 96 incorrect, 157 total)
 c = 3, w = 2 Zero/one-error on test set: 61.78% (60 correct, 97 incorrect, 157 total)
 c = 3, w = 4 Zero/one-error on test set: 61.78% (60 correct, 97 incorrect, 157 total)
 
 Fold 4
 c = default  Zero/one-error on test set: 69.43% (48 correct, 109 incorrect, 157 total)
 c = 10       Zero/one-error on test set: 70.06% (47 correct, 110 incorrect, 157 total)
 c = 5, l = 2 Zero/one-error on test set: 69.43% (48 correct, 109 incorrect, 157 total)
 c = 3, w = 2 Zero/one-error on test set: 68.79% (49 correct, 108 incorrect, 157 total)
 c = 3, w = 4 Zero/one-error on test set: 69.43% (48 correct, 109 incorrect, 157 total)
 
  Fold 5
 c = default  Zero/one-error on test set: 64.33% (56 correct, 101 incorrect, 157 total)
 c = 10       Zero/one-error on test set: 64.97% (55 correct, 102 incorrect, 157 total)
 c = 5, l = 2 Zero/one-error on test set: 63.69% (57 correct, 100 incorrect, 157 total)
 c = 3, w = 2 Zero/one-error on test set: 64.97% (55 correct, 102 incorrect, 157 total)
 c = 3, w = 4 Zero/one-error on test set: 64.97% (55 correct, 102 incorrect, 157 total)
 
 
 
 
The parameter c which is defined as the trade-off between training error and margin is varried from 5 to 20 for each folds. Increase in c , increased the zero/one error on test set averaged over 5 fold. However, when loss function is changed from Total number of swapped pairs summed over all queries to Fraction of swapped pairs averaged over all queries, it is decreased the zero/one error on test set. Another parameter that affected the performance is the choice of structural learning algorithm. The results are compared for  1-slack algorithm (primal) with 1-slack algorithm (dual) with constraint cache. It is found that 1 slack algorith (primal) performs better than the other one, however 1-slack algorithm (dual) with constraint cache performs better than the 1-slack algorithm (dual). 

Average 
c = 20     62.24% 
c = 10     62.62%
c = 5      61.47%
c = 3,w =2 61.85%
c=3, w=4   61.94%





## Part 2.1: Noise! (15 points)

Now we're going to investigate whether the ranking SVM is easily influenced by noisy features. For example, what if some of the features you have are in error? Or what if you downloaded only a portion of a page to calculate a feature? (so the count of inlinks would be wrong)? 

In this case, add some noise to the features. What happens to the results? You may choose to add random noise throughout, noise to a single feature, noise to multiple features, etc. The choices are up to you. We aim to see what kind of exploration you conduct and what you conclude.

*add your results and discussion here*
I have added white IID gaussian noise to the data and the results are 
fold 5 Zero/one-error on test set:73.25% 42 correct 115 incorrect 157 total
fold 4 Zero/one-error on test set:71.97% 44 correct 113 incorrect 157 total
fold 3 Zero/one-error on test set:70.70% 46 correct 111 incorrect 157 total
fold 2 Zero/one-error on test set:64.97% 55 correct 102 incorrect 157 total
fold 1 Zero/one-error on test set:65.38% 54 correct 102 incorrect 156 total

here is the scrip to generate noise
#from random import randint

#fr = open('F:/SEM-2/IR/HW_1/folds/Fold5/train.txt', 'r')
#fw = open('F:/SEM-2/IR/HW_1/folds/Fold5/train.mod.txt', 'w')

#for line in fr:
        #x = line.split()
        #newline = str(x[0]) + ' ' + str(x[1])
        #i = 2
        #while (i != 48):
                #y = x[i].split(':')
                #newline = newline + ' ' + str(y[0]) + ':' + str(float(y[1]) + randint(0,100)/1.0)
                #i = i + 1
        #while (i != 57):
                #newline = newline + ' ' + str(x[i])
                #i = i + 1
        #fw.write(newline)
        #fw.write('\n')


## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*